In [1]:
import numpy as np
from astropy.table import Table, vstack

In [2]:
path = './../catalogs/'
VU_GC = Table.read(path+'VUGC_pPXF_2017_v2.fits')
VD_GC = Table.read(path+'VDGC_pPXF_2017_v2.fits')

In [3]:
columns_to_change= ['ZERR', 'ZERR_ne', 'ZERR_pe', 'ZHEL', 'ZOBS', 'ZOBS_ne', 'ZOBS_pe']

for col in columns_to_change:
    VD_GC[col] = VD_GC[col] * 3e+5
    
    VD_GC.rename_column(col,'V' + col[1:])
    
VU_GC['KECKID'] = '---'
VU_GC['ZQUAL'] = -101010
VU_GC['ZSPECNUM'] = -101010

In [4]:
test_VDGC = VD_GC[VD_GC['ZCONF']==1]
test_VDGC = test_VDGC[test_VDGC['GCSAT']== 'Y']
Host = VU_GC[((VU_GC['HOST']== 'VLSB-B') | (VU_GC['HOST']== 'VLSB-D') | (VU_GC['HOST']=='VCC0615')) & (VU_GC['ZCONF']==1) & (VU_GC['GCSAT']=='Y')]
Not_host = VU_GC[((VU_GC['HOST'] != 'VLSB-B') & (VU_GC['HOST'] != 'VLSB-D') & (VU_GC['HOST'] != 'VCC0615') & (VU_GC['ZCONF']==1) & (VU_GC['GCSAT']=='Y'))]
test_VDGC= vstack([test_VDGC, Not_host])
Host.remove_rows(np.nonzero(Not_host)[0])

In [5]:
test_VDGC.remove_rows(np.nonzero((test_VDGC['VHEL']>-300) & (test_VDGC['VHEL'] < 300)))
Host.remove_rows(np.nonzero((Host['VHEL'] > -300) & (Host['VHEL'] < 300)))

In [6]:
Host

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,MASKNAME,SLITNUM,YLOW,YHIGH,SPEC1DNAME,KECKID,ZQUAL,ZSPECNUM
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes20,int16,int16,int16,bytes40,str3,int64,int64
---,GCXL,Y,VLSB-D,186.17159,13.580055,1006.92,8.453587,4.7,7.19,1.0,987.41,4.7,7.19,-28.51,5.59,6.43,9.0,6.17,vugc5,1,4,7,spec1d.vugc5.001.GCXL001.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18745,13.564,1057.16,10.3895645,7.36,7.03,1.0,1052.7,7.36,7.03,-13.46,7.44,7.55,9.0,6.33,vugc5,2,24,32,spec1d.vugc5.002.GCXL002.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.17741,13.550445,1026.31,4.880886,3.9,4.49,1.0,1028.8,3.9,4.49,-6.51,2.44,2.55,9.0,11.62,vugc5,3,55,63,spec1d.vugc5.003.GCXL003.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18413,13.556723,1056.22,12.89066,15.56,8.47,1.0,1056.32,15.56,8.47,-8.9,4.43,4.91,9.0,2.39,vugc5,4,16,22,spec1d.vugc5.004.GCXL004.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18338,13.483723,1019.73,12.122438,7.14,5.4,1.0,1072.4,7.14,5.4,43.67,13.04,7.71,9.0,5.08,vugc5,6,72,82,spec1d.vugc5.006.GCXL007.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18124,13.522139,1016.17,2.5402215,2.32,2.21,1.0,1037.49,2.32,2.21,12.32,1.11,1.19,9.0,21.19,vugc5,7,44,52,spec1d.vugc5.007.GCXL009.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.15533,13.511806,1030.47,18.865732,9.73,7.2,1.0,1047.31,9.73,7.2,7.84,16.95,16.77,9.0,4.41,vugc5,10,66,78,spec1d.vugc5.010.GCXL013.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.16821,13.523222,1028.09,3.355,3.79,2.92,1.0,1037.09,3.79,2.92,81.95,76.19,81.31,9.0,8.13,vugc5,11,5,13,spec1d.vugc5.011.GCXL014.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.17363,13.5448885,1057.31,8.705144,8.14,6.45,1.0,1051.42,8.14,6.45,-14.89,5.03,4.47,9.0,6.42,vugc5,13,32,40,spec1d.vugc5.013.GCXL017.fits.gz,---,-101010,-101010


In [7]:
fileOrphanDE = Table.read(path + 'orphanGCs.fits')
filekinDE = Table.read(path + 'VDGC_kinematic_prob.fits')
fileOrphanUDG = Table.read(path + 'SelectGCsTrue2.fits')
filekinUDG = Table.read(path + 'SelectGCsTrue_kinematic_prob.fits')

In [8]:
def typeandprob(file):
    probability= []
    objecttype= []
    for row in file:
        probability.append(max(row))
        vals = []
        for ind, val in enumerate(row):
            vals.append(val)
        if (vals.index(max(vals))==0):
            objecttype.append('Star')
        elif (vals.index(max(vals))==1):
            objecttype.append('ICGC')
        elif (vals.index(max(vals))==2):
            objecttype.append('M87GC')
    return probability, objecttype

In [9]:
probabilityDE, objecttypeDE = typeandprob(filekinDE)
probabilityUDG, objecttypeUDG = typeandprob(filekinUDG)


In [10]:
fileOrphanDE['Probability'] = probabilityDE
fileOrphanUDG['Probability'] = probabilityUDG
fileOrphanDE['Type'] = objecttypeDE
fileOrphanUDG['Type'] = objecttypeUDG

In [37]:
noDE = VD_GC[(VD_GC['ZCONF']==1) & (VD_GC['GCSAT']=='N') | (VD_GC['GCSAT']=='M')]
noUDG = VU_GC[(VU_GC['ZCONF']==1) & (VU_GC['GCSAT']=='N') | (VU_GC['GCSAT']=='M')]


In [38]:
noDE.sort('SPEC1DNAME')
noUDG.sort('SPEC1DNAME')
fileOrphanDE.sort('SPEC1DNAME')
fileOrphanUDG.sort('SPEC1DNAME')

focusedUDG = fileOrphanUDG['SPEC1DNAME', 'Probability', 'Type']
focusedUDG.add_row(['spec1d.vugc1.044.GCXO0504.fits.gz', -1, 'Unknown'])
focusedUDG.sort('SPEC1DNAME')

noDE['Probability'] = probabilityDE
noUDG['Probability'] = focusedUDG['Probability']
noDE['Type'] = objecttypeDE
noUDG['Type'] = focusedUDG['Type']

In [40]:
noDE

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME,Probability,Type
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes20,bytes20,int16,int16,int16,int16,int16,bytes40,float32,str5
---,ALG,N,---,187.0097,10.359333,54.56785,2.5169947,1.2761084,1.2873586,1.0,117.61137,0.21014537,0.21014537,0.00016147838,4.6698975e-07,7.3384103e-07,4.866667e-05,196.03,alg42,vdgc1,0,0,-1,15,21,spec1d.vdgc1.000.alg42.fits.gz,0.98157275,Star
---,ALG,N,---,187.07608,10.3505,-210.7357,3.0194468,2.024616,2.1834822,1.0,-170.35785,1.5010384,1.6711562,8.592611e-05,1.7678897e-06,2.1348103e-06,4.866667e-05,58.89,alg115,vdgc1,1,1,-1,15,21,spec1d.vdgc1.001.alg115.fits.gz,0.90411496,Star
---,ALG,N,---,186.9995,10.459528,12.869002,2.8418825,1.7621847,1.9203647,1.0,76.86318,1.0807476,1.2108377,0.00016464724,2.0347409e-06,2.701869e-06,4.866667e-05,44.0,alg140,vdgc1,2,2,-1,15,21,spec1d.vdgc1.002.alg140.fits.gz,0.982906,Star
---,ALG,N,---,187.04004,10.45025,19.96391,2.5854344,1.3975698,1.426399,1.0,32.882748,0.46031845,0.57039464,-5.6038766e-06,1.4009692e-06,1.2675436e-06,4.866667e-05,80.71,alg164,vdgc1,3,3,-1,16,21,spec1d.vdgc1.003.alg164.fits.gz,0.9828415,Star
---,ALG,N,---,187.07025,10.325111,-2.7317898,2.612219,1.4655958,1.453245,1.0,34.784065,0.65045,0.6404431,7.6386175e-05,1.3342564e-06,1.2341872e-06,4.866667e-05,92.15,alg177,vdgc1,4,4,-1,15,20,spec1d.vdgc1.004.alg177.fits.gz,0.98282486,Star
---,ALG,N,---,187.05908,10.544111,17.422152,2.510731,1.271273,1.2664591,1.0,-29.680532,0.17011769,0.14009692,-0.00020567562,5.003461e-07,4.6698975e-07,4.866667e-05,222.86,alg191,vdgc1,5,5,-1,15,21,spec1d.vdgc1.005.alg191.fits.gz,0.9828719,Star
---,ALG,N,---,187.06075,10.386139,-53.867165,2.7268543,1.7217433,1.5956256,1.0,-18.522814,0.73050535,0.70048463,6.9147834e-05,3.1021461e-06,2.3349487e-06,4.866667e-05,73.2,alg194,vdgc1,6,6,-1,15,20,spec1d.vdgc1.006.alg194.fits.gz,0.9802294,Star
VCC1075,NUC,N,---,187.05128,10.297639,1811.7435,2.917756,1.9805114,1.9400272,1.0,1825.7831,1.4409969,1.2608724,-1.867959e-06,1.7678897e-06,2.6018e-06,4.866667e-05,28.58,nuc5,vdgc1,7,7,4,30,37,spec1d.vdgc1.007.nuc5.fits.gz,1.0,ICGC
VCC1079,NUC,N,---,187.05008,10.365334,1334.1931,3.0186136,2.089962,2.114991,1.0,1349.0332,1.5510731,1.541066,8.0055383e-07,2.101454e-06,2.435018e-06,4.866667e-05,32.52,nuc6,vdgc1,8,8,4,19,25,spec1d.vdgc1.008.nuc6.fits.gz,1.0,ICGC
